In [1]:

import os

os.environ["R_HOME"] = r"C:\Program Files\R\R-4.1.1"
os.environ["PATH"]   = r"C:\Program Files\R\R-4.1.1\bin\x64" + ";" + os.environ["PATH"]

import pyper as pr
import torch
import sklearn as sl
import numpy as np
from torch import nn
import torch.nn.functional as F
import statsmodels.api as sm
import pandas as pd

r = pr.R(use_pandas = True)
#r("install.packages('Gammareg')")
r("library(Gammareg)")
#r("install.packages('dglm')")
r("library(dglm)")

"try({library(dglm)})\r\nWarning message:\r\npackage 'dglm' was built under R version 4.1.3 \r\n"

## Data Simulation
We simulate two risk classes. We assume the probability of a claim is 100%. We simulate parameters to be sampled from a Gamma distribution. Parameters are randomly sampled by samples in risk characteristics as follows:

$ X_{3} $ indicates a high risk insured.

$\hat{\theta}_{low} = 1 + \beta_{1} X_{1} + \beta_{2} X_{2} $

$\hat{\theta}_{high} = 2 + 1.25 \beta_{1} X_{1} + 1.25 \beta_{2} X_{2} + \beta_{3} X_{3}$

$\hat{\alpha}_{low} = 1 + 0.1 \beta_{1} X_{1} $

$\hat{\alpha}_{high} = 1 + 0.2 \beta_{1} X_{1} $

**Where samples from each class are defined as:**

$\hat{Y}_{low} \:$~$\: Gamma(\alpha = \hat{\alpha}_{low}, \theta = \hat{\theta}_{low}) $

$\hat{Y}_{high} \:$~$\: Gamma(\alpha = \hat{\alpha}_{high}, \theta = \hat{\theta}_{high}) $

**Our characteristic distribution is as follows:**

$X_{1},X_{2} \:$~$\: Normal(\mu = 5, \sigma = 1) $

$\beta_{1}, \beta_{2}, \beta_{3} \:$~$\: Normal(\mu = 5, \sigma = 0.5) $

In [2]:
from functions import SimulateRisk

low_risk = SimulateRisk(True, 100)
high_risk = SimulateRisk(False, 100)

low_risk_samples = low_risk.getData()
high_risk_samples = high_risk.getData()

total_samples = pd.concat([low_risk_samples, high_risk_samples], ignore_index = True)

low_risk_expected = low_risk.getExpected()
high_risk_expected = high_risk.getExpected()

#Seperate samples from known risk characteristics
Y_low = low_risk_samples['Response'].to_frame()
X_low = low_risk_samples.drop('Response', axis=1)

Y_high = high_risk_samples['Response'].to_frame()
X_high = high_risk_samples.drop('Response', axis=1)

Y_total = pd.concat([Y_low, Y_high], ignore_index=True)
X_total = pd.concat([X_low, X_high], ignore_index=True)
Expected_total = pd.concat([low_risk_expected, high_risk_expected], ignore_index=True)

In [3]:
#Assign our R stuff here
r.assign("Y", Y_total['Response'].to_list())
r.assign("X0", X_total['x1'].to_list())
r.assign("X1", X_total['x1'].to_list())
r.assign("X2", X_total['x2'].to_list())
r.assign("X3", X_total['x3'].to_list())
r.assign("data", total_samples)
r("X <- cbind(X0, X1, X2)")
r("Z <- cbind(X0, X3)")
r("formula.mean = Response ~ x1 + x2")
r("formula.dispersion = ~ x3")

'try({formula.dispersion = ~ x3})\r\n'

In [4]:
u = r.get("Y")
np.max(u)


0.437574577267581

In [5]:
from functions import sampleDiagnositcs

sampleDiagnositcs(Y_low)
print("")
sampleDiagnositcs(Y_high)


Here are the requested diagnostics:
mean: 0.05
std: 0.05
min: 0.00
q1%: 0.00
q25%: 0.01
q75%: 0.01
q99%: 0.21
max: 0.35

Here are the requested diagnostics:
mean: 0.14
std: 0.09
min: 0.00
q1%: 0.02
q25%: 0.07
q75%: 0.07
q99%: 0.42
max: 0.44


**Data Transform**

In [6]:
from sklearn.linear_model import GammaRegressor
r("library(Gammareg)")

model = GammaRegressor(solver = 'newton-cholesky')
model.fit(X = X_total, y = Y_total)

model_n = sm.GLM(Y_total, X_total, family = sm.families.Gamma(link=sm.families.links.Log())).fit()
#r('a = Gammareg(Y~X1+X2, ~X3, meanlink="log")')
r('model = dglm(formula = formula.mean, dformula = formula.dispersion, family = Gamma(link = "log"), dlink = "log", data = data)')


C:\Users\Luke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Luke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Luke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\e

'try({model = dglm(formula = formula.mean, dformula = formula.dispersion, family = Gamma(link = "log"), dlink = "log", data = data)})\r\n'

In [7]:
print(r('summary(model)'))

try({summary(model)})

Call: dglm(formula = formula.mean, dformula = formula.dispersion, family = Gamma(link = "log"), 
    dlink = "log", data = data)

Mean Coefficients:
               Estimate  Std. Error    t value     Pr(>|t|)
(Intercept) -3.58846377 0.338473810 -10.601895 4.559697e-21
x1           0.02647007 0.006312014   4.193601 4.147374e-05
x2           0.06652015 0.019436498   3.422435 7.547396e-04
(Dispersion Parameters for Gamma family estimated as below )

    Scaled Null Deviance: 261.4161 on 199 degrees of freedom
Scaled Residual Deviance: 228.0523 on 197 degrees of freedom

Dispersion Coefficients:
              Estimate Std. Error   z value     Pr(>|z|)
(Intercept)  0.4016719  0.1199906  3.347527 8.153604e-04
x3          -1.3420138  0.1793066 -7.484462 7.184077e-14
(Dispersion parameter for Digamma family taken to be 2 )

    Scaled Null Deviance: 292.2948 on 199 degrees of freedom
Scaled Residual Deviance: 243.2938 on 198 degrees of freedom

Minus Twice the Log-Likeli

In [8]:
print(r('summary(a)'))

try({summary(a)})
Error in summary(a) : object 'a' not found



In [9]:
model_n.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Response   No. Observations:                  200
Model:                            GLM   Df Residuals:                      196
Model Family:                   Gamma   Df Model:                            3
Link Function:                    Log   Scale:                         0.65916
Method:                          IRLS   Log-Likelihood:                 314.08
Date:                Sat, 11 Nov 2023   Deviance:                       163.23
Time:                        18:09:26   Pearson chi2:                     129.
No. Iterations:                    13   Pseudo R-squ. (CS):             0.4195
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x0            -4.2740      0.327    -13.062      0.000      -4.915      -3.633
x1             0.0296      0.007      4.503      0.000       0.017       0.043
x2             0.0511      0.020      2.516      0.012       0.011       0.091
x3             1.0103      0.115      8.762      0.000       0.784       1.236
==============================================================================
"""

In [10]:
from functions import SimulatePortfolio

portfolio = SimulatePortfolio(iterations = 100, profit = 0.05, size = 1000)
portfolio.runSimulation()
portfolio.runSimulationAdj()

100 iterations complete, 0 remaining.
100 iterations complete, 0 remaining.


In [11]:
h = portfolio.getHighMeanPercent()

In [12]:
l = portfolio.getLowMeanPercent()

In [13]:
t = portfolio.getTotalMeanPercent()

In [14]:
np.mean(l)

0.03462416415622792

In [15]:
np.mean(t)

0.02801749958258906

In [16]:
np.mean(h)

0.025603393611053526

In [17]:
print(np.mean(portfolio.theoreticalProfit))
print(np.mean(portfolio.sampledProfit))

0.04999512289945326
0.04946991511131256


In [18]:
print(np.mean(portfolio.theoreticalProfitAdj))
print(np.mean(portfolio.sampledProfitAdj))

0.0494470438824104
0.04770954531293382


Sharpe ratio for profit

In [19]:
np.mean(portfolio.theoreticalProfitAdj)/np.std(portfolio.theoreticalProfitAdj, ddof=1)

2.8772598147391255

In [20]:
np.mean(portfolio.theoreticalProfit)/np.std(portfolio.theoreticalProfit, ddof=1)

3.1383762860448186

In [21]:
np.std(portfolio.theoreticalProfit, ddof=1)

0.015930251296430834

In [22]:
np.std(portfolio.theoreticalProfitAdj, ddof=1)

0.017185463623796395

In [23]:
print(np.mean(portfolio.theoreticalLowProfitAdj))
print(np.mean(portfolio.theoreticalLowProfit))

0.06482141055024661
0.050459381795895944


In [24]:
print(np.std(portfolio.theoreticalLowProfitAdj, ddof=1))
print(np.std(portfolio.theoreticalLowProfit, ddof=1))

0.030067833126809107
0.03292064910821297


In [25]:
print(np.mean(portfolio.theoreticalHighProfitAdj))
print(np.mean(portfolio.theoreticalHighProfit))

0.043247416496321474
0.04938018782063788


In [26]:
print(np.std(portfolio.theoreticalHighProfitAdj, ddof=1))
print(np.std(portfolio.theoreticalHighProfit, ddof=1))

0.02135925524579955
0.01909984834890106


In [27]:
np.std(portfolio.theoreticalHighProfit, ddof=1)/np.std(portfolio.theoreticalLowProfit, ddof=1)

0.580178364227213

In [28]:
np.std(h)

0.011152195721302542

In [29]:
np.std(l)

0.017051816216649245

In [30]:
portfolio.highAdj

0.04569178904702165

In [31]:
portfolio.lowAdj

0.06179024658171931

Here we demonstrate the constant pricing:

In [32]:
portfolio.highAdjOB/np.mean(h)

1.7845989379820157

In [33]:
portfolio.lowAdjOB/np.mean(l)

1.7845989379820155

In [34]:
portfolio.theoreticalProfit

[0.05534925155547421,
 0.05346974752193345,
 0.04869749506789345,
 0.08148168869878392,
 0.06384513255418389,
 0.07985206874259909,
 0.045176247198099095,
 0.05237549079824133,
 0.04758677388364729,
 0.06070812146746152,
 0.05949964817084619,
 0.05139440808878637,
 0.03849256031097481,
 0.03670181073133982,
 0.047016372264699346,
 0.041163074840897984,
 0.04078670324681577,
 0.05387391206227865,
 0.05394839017261366,
 0.06345874779230554,
 0.06488747145174556,
 0.05428481786220585,
 0.05259241869398901,
 0.06697557353010153,
 0.04313248885122245,
 0.03114175220709514,
 0.0472162476069129,
 0.030439744219035725,
 0.06619500832953784,
 0.049589975457708824,
 0.06228713502107175,
 0.0261216713900444,
 0.04720271890359229,
 0.10009995874624567,
 0.02308770650313452,
 0.03286977006150238,
 0.06956924505878348,
 0.03980892478751885,
 0.03637845217323976,
 0.030814907944205938,
 0.053660267761240776,
 0.05238961121804697,
 0.03549833783687695,
 0.04906286913075586,
 0.07500510597253862,
 0.05

In [35]:
portfolio.theoreticalProfitAdj

[0.04312001375541474,
 0.05413804984850834,
 0.04842584836510033,
 0.03745101728617539,
 0.05466697356551742,
 0.04942095884208919,
 0.03664862764473842,
 0.0502862520115841,
 0.0386140987578133,
 0.050190025619943346,
 0.03178708495202154,
 0.06501430381028905,
 0.04894824220657079,
 0.05238445002480008,
 0.06154562747146619,
 0.06657683180164398,
 0.02766788061893488,
 0.05901197131432734,
 0.05120342178603243,
 0.06910349592797316,
 0.07212629493238387,
 0.05535030048186429,
 0.06493643099549273,
 0.055681434188586176,
 0.05282203780365913,
 0.05565973191729878,
 0.04527566784461656,
 0.07364578264833743,
 0.03916333398562433,
 0.05557061011241038,
 0.04835193577021757,
 0.060502156837908494,
 0.0651279038927638,
 0.039760399445391004,
 0.0918888331722526,
 0.07453504084709961,
 0.05964731009168067,
 0.047898573842167114,
 0.07256899652008131,
 0.0629987702582483,
 0.0486346993082436,
 0.05058756215014226,
 0.053186807851553386,
 0.05874887325259581,
 0.05238227231200854,
 0.0263104

In [36]:
portfolio.highAdj/portfolio.lowAdj

0.7394660415635822

Disparity in profit adj correlates with size of portfolio. As n grows, the risk flips in the sense that high is over priced and low is under priced. For sufficiently low n, high is under priced and low is over priced.

In [37]:
portfolio.getSigmaHighList()

In [38]:
portfolio.getSigmaLowList()

## Modelling

In [39]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [40]:
from functions import NeuralNetworkBinary
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

#We need to first rebalance the transformed dataset. (can't use weights due to loss function not supporting)
data_class_1 = transformed_data[transformed_data.overall_positive==1]
data_class_0 = transformed_data[transformed_data.overall_positive==0]

class_1_resampled = resample(data_class_1, replace=False, n_samples=len(data_class_0), random_state=123)
transformed_data = pd.concat([data_class_0, class_1_resampled])

#Create data frames for x and y
response_var_binary = transformed_data['overall_positive']
predictor_var = transformed_data.drop(['overall_positive', 'overall'], axis = 1)

#Create test and train datasets
X_train_binary, X_test_binary, Y_train_binary, Y_test_binary = train_test_split(predictor_var, response_var_binary, test_size=0.5, random_state=42, stratify=response_var_binary)

#Create tensors
response_var_binary_t = torch.tensor(Y_train_binary.values, dtype = torch.float32).to(device)
predictor_var_t = torch.tensor(X_train_binary.values, dtype = torch.float32).to(device)

#Create model
n_columns = len(transformed_data.columns)-2

model = NeuralNetworkBinary(n_input = n_columns, n_hidden_layer = n_columns, n_output = 1, learning_rate = 0.0075).to(device)
print(model)

ImportError: cannot import name 'NeuralNetworkBinary' from 'functions' (c:\Users\Luke\MyRepo\MissPricing\functions.py)

In [ ]:
#Convert test data to tensors
Y_test_binary_t = torch.tensor(Y_test_binary.values, dtype = torch.float32).to(device)
X_test_binary_t = torch.tensor(X_test_binary.values, dtype = torch.float32).to(device)

model.train(x_train = predictor_var_t, y_train = response_var_binary_t, batch_size=6000, x_test = X_test_binary_t, y_test = Y_test_binary_t)

## Model Diagnostics ##

In [ ]:
from utils import accuracy

X_test_binary_t = torch.tensor(X_test_binary.values, dtype = torch.float32).to(device)

predictions_binary = model.model(X_test_binary_t).detach().cpu().numpy()
#Turn probability into prediction
predictions_binary = np.where(predictions_binary > 0.5, 1, 0)

accuracy(Y_test_binary, predictions_binary)

In [ ]:
from utils import confusionMatrix

confusionMatrix(Y_test_binary, predictions_binary)